In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
import regex as re
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
gensim_words = [[w for w in re.split(r'[\\|\\#]', doc.lower()) if w not in stop_words]
                    for doc in df["nav"]]

In [ ]:
from gensim.models import FastText
ft = FastText(gensim_words, min_count=5)
ft.wv.save_word2vec_format("heise-articles-2020.ft")

In [ ]:
ft.wv.most_similar(positive=["java"], topn=10)

In [ ]:
ft.wv.most_similar(positive=["microsoft"], topn=10)

In [ ]:
ft.wv.most_similar(positive=["corona"], topn=10)

In [ ]:
pd.DataFrame(ft.wv.most_similar(positive=["apple", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
pd.DataFrame(ft.wv.most_similar(positive=["microsoft", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
open("heise-articles-nav-2020.txt", "w").write("\n".join([n.lower() 
                                                          for n  in df["nav"].str.replace(r'\#|\|', ' ').values]))

In [ ]:
if ON_COLAB:
    os.system("heise-articles-2020.glove.txt || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.glove.txt.gz && gunzip heise-articles-2020.glove.txt.gz")
    glove_file = 'heise-articles-2020.glove.txt'
else:
    glove_file = '../99_Common/heise-articles-2020.glove.txt'

In [ ]:
from gensim.models import KeyedVectors
glove = KeyedVectors.load_word2vec_format(glove_file, no_header=True)

In [ ]:
glove.most_similar(positive=["java"], topn=10)

In [ ]:
glove.most_similar(positive=["microsoft"], topn=10)

In [ ]:
glove.most_similar(positive=["corona"], topn=10)

In [ ]:
pd.DataFrame(glove.most_similar(positive=["apple", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
pd.DataFrame(glove.most_similar(positive=["microsoft", "android"], 
                                   negative=["google"],  topn=10))